In [1]:
!pip install efficientnet
!pip install transformers==4.6.1
# !pip install huggingface_hub
# !pip install git+https://github.com/Maluuba/nlg-eval.git@master

In [2]:
!pip install pymc3==3.10.0

  Using cached pymc3-3.10.0-py3-none-any.whl (1.9 MB)
  Using cached patsy-0.5.2-py2.py3-none-any.whl (233 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached Theano-PyMC-1.0.11.tar.gz (2.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached arviz-0.12.1-py3-none-any.whl (1.6 MB)
  Using cached fastprogress-1.0.3-py3-none-any.whl (12 kB)
  Using cached xarray_einstats-0.2.2-py3-none-any.whl (33 kB)
  Using cached xarray-0.20.2-py3-none-any.whl (845 kB)
  Using cached netCDF4-1.6.0.tar.gz (774 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [29 lines of output]
  Traceback (most recent call last):
    File "<string>", line 36, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\91833\AppData\Local\Temp\pip-install-2co473nd\netcdf4_c496210d352749379b08b27273eb388d\setup.py", line 444, in <module>
      _populate_hdf5_info(dirstosearch, inc_dirs, libs, lib_dirs)
    File "C:\Users\91833\AppData\Local\Temp\pip-install-2co473nd\netcdf4_c496210d352749379b08b27273eb388d\setup.py", line 385, in _populate_hdf5_info
      raise ValueError('did not find HDF5 headers')
  ValueError: did not find HDF5 headers
  reading from setup.cfg...
  
      HDF5_DIR environment variable not set, checking some standard locations ..
  checking C:\Anaconda\envs\RS1\include ...
  hdf5 headers not found in C:\Anaconda\envs\RS1\include
  checking C:\Anaconda\envs\RS1\Library\include ...
  hdf5 headers no

In [3]:
import tensorflow as tf
from CNN_encoder import CNN_Encoder
from configs import argHandler
import time
from medical_w2v_wrapper import Medical_W2V_Wrapper
from tokenizer_wrapper import TokenizerWrapper
import matplotlib.pyplot as plt
from utility import get_optimizer, get_enqueuer
import os
import json
from augmenter import augmenter
from gpt.gpt2_model import TFGPT2LMHeadModel
from testy import evaluate_enqueuer
import pandas as pd
from glob import glob
import shutil

C:\Anaconda\envs\RS1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
FLAGS = argHandler()
FLAGS.setDefaults()
tf.keras.backend.set_learning_phase(1)

tokenizer_wrapper = TokenizerWrapper(FLAGS.all_data_csv, FLAGS.csv_label_columns[0],
                                     FLAGS.max_sequence_length, FLAGS.tokenizer_vocab_size)

train_enqueuer, train_steps = get_enqueuer(FLAGS.train_csv, FLAGS.batch_size, FLAGS, tokenizer_wrapper)
test_enqueuer, test_steps = get_enqueuer(FLAGS.test_csv, 1, FLAGS, tokenizer_wrapper)
batch_test_enqueuer, batch_test_steps = get_enqueuer(FLAGS.test_csv, FLAGS.batch_size, FLAGS, tokenizer_wrapper)

train_enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)

medical_w2v = Medical_W2V_Wrapper()
# medical_w2v.save_embeddings(tokenizer_wrapper.get_word_tokens_list(),FLAGS.tags)
# embeddings = medical_w2v.get_embeddings_matrix_for_words(tokenizer_wrapper.get_word_tokens_list(),
#                                                          FLAGS.tokenizer_vocab_size)
tags_embeddings = medical_w2v.get_embeddings_matrix_for_tags(FLAGS.tags)
# print(f"Embeddings shape: {embeddings.shape}")
print(f"Tags Embeddings shape: {tags_embeddings.shape}")

del medical_w2v

encoder = CNN_Encoder('pretrained_visual_model', FLAGS.visual_model_name, FLAGS.visual_model_pop_layers,
                      FLAGS.encoder_layers,
                      FLAGS.tags_threshold, tags_embeddings, FLAGS.finetune_visual_model, len(FLAGS.tags))

decoder = TFGPT2LMHeadModel.from_pretrained('distilgpt2', from_pt=True, resume_download=True)
optimizer = get_optimizer(FLAGS.optimizer_type, FLAGS.learning_rate)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, tokenizer_wrapper.GPT2_pad_token_id()))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


loss_plot = []


#
@tf.function
def train_step(images, target, test_mode=False):
    with tf.GradientTape() as tape:
        visual_features, tags_embeddings = encoder(images)
        dec_input = target[:, 0:-1]

        # passing the features through the decoder
        predictions, _ = decoder(dec_input, visual_features=visual_features, tags_embeddings=tags_embeddings, past=None)

        loss = loss_function(target[:, 1:], predictions)
    if not test_mode:
        trainable_variables = encoder.trainable_variables + decoder.trainable_variables
        gradients = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss


ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)

try:
    os.makedirs(os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
except:
    print("path already exists")

with open(os.path.join(FLAGS.ckpt_path, 'configs.json'), 'w') as fp:
    json.dump(FLAGS, fp, indent=4)

ckpt_manager = tf.train.CheckpointManager(ckpt, FLAGS.ckpt_path, max_to_keep=5)

start_epoch = 0
best_test_avg_score = 0


def get_avg_score(scores_dict):
    avg_score = 0
    for value in scores_dict.values():
        avg_score += value
    avg_score = avg_score / len(scores_dict)
    return avg_score


if ckpt_manager.latest_checkpoint and FLAGS.continue_from_last_ckpt:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Restored from checkpoint: {}".format(ckpt_manager.latest_checkpoint))
    try:
        with open(os.path.join(FLAGS.ckpt_path, 'scores.json')) as scores_file:
            scores = json.load(scores_file)
            best_test_avg_score = get_avg_score(scores)
            print(f"best scores: {scores}")
    except:
        print("No previous scores found")

train_batch_losses_csv = {"step": [], "batch_loss": []}
test_batch_losses_csv = {"step": [], "batch_loss": []}
train_after_batch_losses_csv = {"step": [], "batch_loss": []}
losses_csv = {"epoch": [], "train_loss": [], "train_after_loss": [], "test_loss": []}
time_csv = {"epoch": [], 'time_taken': [], "scores": []}


def get_overall_loss(enqueuer, steps, batch_losses_csv):
    tf.keras.backend.set_learning_phase(0)

    if not enqueuer.is_running():
        enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)
    generator = enqueuer.get()

    batch_losses = []
    total_loss = 0
    step = 0
    for batch in range(steps):
        img, target, _ = next(generator)
        batch_loss = train_step(img, target, True)
        batch_losses_csv['step'].append(step)
        batch_losses_csv['batch_loss'].append(batch_loss.numpy())
        total_loss += batch_loss
        batch_losses.append(batch_loss)
        step += 1
    epoch_loss = total_loss / generator.steps
    enqueuer.stop()
    tf.keras.backend.set_learning_phase(1)

    return epoch_loss, batch_losses


train_generator = train_enqueuer.get()

for epoch in range(start_epoch, FLAGS.num_epochs):
    start = time.time()
    total_loss = 0
    times_to_get_batch = 0
    pure_training_time = 0
    step = 0
    for batch in range(train_steps):
        t = time.time()
        img, target, _ = next(train_generator)
        # print("Time to get batch: {} s ".format(time.time() - t))
        if time.time() - t > 2:
            times_to_get_batch += 1
        step_time = time.time()
        batch_loss = train_step(img, target)
        pure_training_time += time.time() - step_time
        total_loss += batch_loss
        step += 1
        train_batch_losses_csv['step'].append(step)
        train_batch_losses_csv['batch_loss'].append(batch_loss.numpy())

        # print("Time to train step: {} s ".format(time.time() - t))

#         if batch % 1 == 0 and batch > 0:
#             print('Epoch {} Batch {} Loss {:.4f}'.format(
#                 epoch + 1, batch, batch_loss.numpy()))
    # storing the epoch end loss value to plot later'
    total_loss = (total_loss / train_steps).numpy()
    loss_plot.append(total_loss)
    print('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                        total_loss))
    print('Time taken for {} epoch {} sec\n'.format(epoch + 1, time.time() - start))
    print('Batches that took long: {}'.format(times_to_get_batch))
    if FLAGS.calculate_loss_after_epoch:
        test_epoch_loss, _ = get_overall_loss(batch_test_enqueuer, batch_test_steps, test_batch_losses_csv)
        train_epoch_loss, _ = get_overall_loss(train_enqueuer, train_steps, train_after_batch_losses_csv)
        losses_csv['train_after_loss'].append(train_epoch_loss.numpy())
        losses_csv['test_loss'].append(test_epoch_loss.numpy())
    else:
        losses_csv['train_after_loss'].append('-')
        losses_csv['test_loss'].append('-')
    losses_csv["epoch"].append(epoch + 1)
    losses_csv['train_loss'].append(total_loss)

    pd.DataFrame(losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'losses.csv'), index=False)
    pd.DataFrame(train_batch_losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'train_batch_losses.csv'), index=False)
    pd.DataFrame(train_after_batch_losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'train_after_batch_losses.csv'),
                                                      index=False)
    pd.DataFrame(test_batch_losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'test_batch_losses.csv'), index=False)
    ckpt_manager.save()

    plt.plot(loss_plot)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Plot')
    plt.savefig(FLAGS.ckpt_path + "/loss.png")

    if epoch % FLAGS.epochs_to_evaluate == 0 and epoch > 0:
        current_avg_score = 0
        print("Evaluating on test set..")
        train_enqueuer.stop()
        current_scores = evaluate_enqueuer(test_enqueuer, test_steps, FLAGS, encoder, decoder, tokenizer_wrapper)
        time_csv['epoch'].append(epoch + 1)
        time_csv['time_taken'].append(pure_training_time)
        time_csv['scores'].append(current_scores)
        df = pd.DataFrame(time_csv)
        df.to_csv(os.path.join(FLAGS.ckpt_path, 'time.csv'), index=False)
        current_avg_score = get_avg_score(current_scores)
        train_enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)
        if best_test_avg_score == 0 or current_avg_score > best_test_avg_score:
            print(f"found a new best model and saving the ckpt")
            shutil.rmtree(os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
            os.mkdir(os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
            for filename in glob(os.path.join(FLAGS.ckpt_path, '*')):
                if os.path.isfile(filename):
                    shutil.copy(filename, os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
            best_test_avg_score = current_avg_score


train_enqueuer.stop()
# plt.show()


Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
Tokenizing dataset..
number of tokens: {'UNK': 1, 'no': 2, 'the': 3, 'is': 4, 'are': 5, 'normal': 6, 'startseq': 7, 'endseq': 8, 'xxxx': 9, 'of': 10, 'and': 11, 'acute': 12, 'pleural': 13, 'or': 14, 'pneumothorax': 15, 'effusion': 16, 'there': 17, 'heart': 18, 'lungs': 19, 'size': 20, 'focal': 21, 'clear': 22, 'pulmonary': 23, 'within': 24, 'in': 25, 'disease': 26, 'cardiopulmonary': 27, 'limits': 28, 'right': 29, 'left': 30, 'consolidation': 31, 'abnormality': 32, 'with': 33, 'silhouette': 34, 'airspace': 35, 'mediastinal': 36, 'lung': 37, 'stable': 38, 'cardiomediastinal': 39, '1': 40, 'changes': 41, 'chest': 42, 'findings': 43, 'spine': 44, 'evidence': 45, 'a': 46, 'mild': 47, 'unremarkable': 48, 'mediastinum': 49, 'contour': 50, 'thoracic': 51, 'without': 52, 'structures': 53, 'for': 54, 'to': 55, 'contours': 56, 'bony': 57, 'effusions': 58, 'seen': 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Anaconda\envs\RS1\lib\site-packages\transformers\tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Tags Embeddings shape: (105, 400)
Loaded model from disk
path already exists
Restored from checkpoint: ./checkpoints/CDGPT2/ckpt-50
No previous scores found


In [5]:
import tensorflow as tf
from CNN_encoder import CNN_Encoder
import os
from configs import argHandler
from tokenizer_wrapper import TokenizerWrapper
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from caption_evaluation import get_evalutation_scores
from utility import get_enqueuer
import numpy as np
from PIL import Image
import json
import time
from gpt.gpt2_model import TFGPT2LMHeadModel
import pandas as pd
from tqdm import tqdm

# input_ids=None,
# visual_features = None,
# tags_embedding = None,
# max_length=None,
# min_length=None,
# do_sample=True,
# early_stopping=False,
# num_beams=None,
# temperature=None,
# top_k=None,
# top_p=None,
# repetition_penalty=None,
# bos_token_id=None,
# pad_token_id=None,
# eos_token_ids=None,
# length_penalty=None,
# no_repeat_ngram_size=None,
# num_return_sequences=None,
# attention_mask=None,
avg_time = 0
step_n = 1


def evaluate_full(FLAGS, encoder, decoder, tokenizer_wrapper, images):
    global avg_time, step_n
    visual_features, tags_embeddings = encoder(images)
    dec_input = tf.expand_dims(tokenizer_wrapper.GPT2_encode("startseq", pad=False), 0)
    # dec_input = tf.tile(dec_input,[images.shape[0],1])
    num_beams = FLAGS.beam_width

    visual_features = tf.tile(visual_features, [num_beams, 1, 1])
    tags_embeddings = tf.tile(tags_embeddings, [num_beams, 1, 1])
    start_time = time.time()
    tokens = decoder.generate(dec_input, max_length=FLAGS.max_sequence_length, num_beams=num_beams, min_length=3,
                              eos_token_ids=tokenizer_wrapper.GPT2_eos_token_id(), no_repeat_ngram_size=0,
                              visual_features=visual_features,
                              tags_embedding=tags_embeddings, do_sample=False, early_stopping=True)
    end_time = time.time() - start_time
    # print(f"Step time: {end_time}")
    avg_time += end_time
    # print(f"avg Step time: {avg_time / step_n}")

    sentence = tokenizer_wrapper.GPT2_decode(tokens[0])
    sentence = tokenizer_wrapper.filter_special_words(sentence)
    step_n += 1
    return sentence


def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result // 2, len_result // 2, l + 1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()


def save_output_prediction(FLAGS, img_name, target_sentence, predicted_sentence):
    if not os.path.exists(FLAGS.output_images_folder):
        os.makedirs(FLAGS.output_images_folder)

    image_path = os.path.join(FLAGS.image_directory, img_name)

    img = mpimg.imread(os.path.join(image_path))

    caption = "Real caption: {}\n\nPrediction: {}".format(target_sentence, predicted_sentence)
    # plt.ioff()
    fig = plt.figure(figsize=(7.20, 10.80))
    fig.add_axes((.0, .5, .9, .7))
    fig.text(.1, .3, caption, wrap=True, fontsize=20)

    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    plt.savefig(FLAGS.output_images_folder + "/{}".format(img_name))
    plt.close(fig)


def evaluate_enqueuer(enqueuer, steps, FLAGS, encoder, decoder, tokenizer_wrapper, name='Test set', verbose=True,
                      write_json=True, write_images=False, test_mode=False):
    tf.keras.backend.set_learning_phase(0)
    hypothesis = []
    references = []
    if not enqueuer.is_running():
        enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)
    start = time.time()
    csv_dict = {"image_path": [], "real": [], "prediction": []}
    generator = enqueuer.get()
    for batch in tqdm(list(range(steps))):
        images, target, img_path = next(generator)
        predicted_sentence = evaluate_full(FLAGS, encoder, decoder, tokenizer_wrapper,
                                           images)
        csv_dict["prediction"].append(predicted_sentence)
        csv_dict["image_path"].append(os.path.basename(img_path[0]))
        target_sentence = tokenizer_wrapper.GPT2_decode(target[0])
        target_sentence = tokenizer_wrapper.filter_special_words(target_sentence)
        csv_dict["real"].append(target_sentence)
        target_word_list = tokenizer_wrapper.GPT2_format_output(target_sentence)
        references.append([target_word_list])
        hypothesis_word_list = tokenizer_wrapper.GPT2_format_output(predicted_sentence)
        if hypothesis_word_list[-1] == hypothesis_word_list[-1]:
            hypothesis_word_list = hypothesis_word_list[:-1]
        hypothesis.append(hypothesis_word_list)
        if write_images:
            save_output_prediction(FLAGS, img_path[0], target_sentence, predicted_sentence)
        # print('Time taken for saving image {} sec\n'.format(time.time() - t))

    enqueuer.stop()
    scores = get_evalutation_scores(hypothesis, references, test_mode)
    print("{} scores: {}".format(name, scores))
    if write_json:
        with open(os.path.join(FLAGS.ckpt_path, 'scores.json'), 'w') as fp:
            json.dump(str(scores), fp, indent=4)
    print('Time taken for evaluation {} sec\n'.format(time.time() - start))
    tf.keras.backend.set_learning_phase(1)
    df = pd.DataFrame(csv_dict)
    df.to_csv(FLAGS.ckpt_path + "/predictions.csv", index=False)
    return scores


if __name__ == "__main__":
    FLAGS = argHandler()
    FLAGS.setDefaults()

    tokenizer_wrapper = TokenizerWrapper(FLAGS.all_data_csv, FLAGS.csv_label_columns[0],
                                         FLAGS.max_sequence_length, FLAGS.tokenizer_vocab_size)

    print("** load test generator **")

    test_enqueuer, test_steps = get_enqueuer(FLAGS.test_csv, 1, FLAGS, tokenizer_wrapper)
    test_enqueuer.start(workers=1, max_queue_size=8)

    encoder = CNN_Encoder('pretrained_visual_model', FLAGS.visual_model_name, FLAGS.visual_model_pop_layers,
                          FLAGS.encoder_layers, FLAGS.tags_threshold, num_tags=len(FLAGS.tags))

    decoder = TFGPT2LMHeadModel.from_pretrained('distilgpt2', from_pt=True, resume_download=True)

    optimizer = tf.keras.optimizers.Adam()

    ckpt = tf.train.Checkpoint(encoder=encoder,
                               decoder=decoder,
                               optimizer=optimizer)

    ckpt_manager = tf.train.CheckpointManager(ckpt, FLAGS.ckpt_path, max_to_keep=1)

    if ckpt_manager.latest_checkpoint:
        start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print("Restored from checkpoint: {}".format(ckpt_manager.latest_checkpoint))
    evaluate_enqueuer(test_enqueuer, test_steps, FLAGS, encoder, decoder, tokenizer_wrapper, write_images=True, test_mode=True)


Tokenizing dataset..
number of tokens: {'UNK': 1, 'no': 2, 'the': 3, 'is': 4, 'are': 5, 'normal': 6, 'startseq': 7, 'endseq': 8, 'xxxx': 9, 'of': 10, 'and': 11, 'acute': 12, 'pleural': 13, 'or': 14, 'pneumothorax': 15, 'effusion': 16, 'there': 17, 'heart': 18, 'lungs': 19, 'size': 20, 'focal': 21, 'clear': 22, 'pulmonary': 23, 'within': 24, 'in': 25, 'disease': 26, 'cardiopulmonary': 27, 'limits': 28, 'right': 29, 'left': 30, 'consolidation': 31, 'abnormality': 32, 'with': 33, 'silhouette': 34, 'airspace': 35, 'mediastinal': 36, 'lung': 37, 'stable': 38, 'cardiomediastinal': 39, '1': 40, 'changes': 41, 'chest': 42, 'findings': 43, 'spine': 44, 'evidence': 45, 'a': 46, 'mild': 47, 'unremarkable': 48, 'mediastinum': 49, 'contour': 50, 'thoracic': 51, 'without': 52, 'structures': 53, 'for': 54, 'to': 55, 'contours': 56, 'bony': 57, 'effusions': 58, 'seen': 59, 'atelectasis': 60, 'degenerative': 61, 'opacity': 62, 'large': 63, 'upper': 64, 'calcified': 65, 'lobe': 66, 'cardiac': 67, '2': 6

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


** load test generator **
Loaded model from disk


Restored from checkpoint: ./checkpoints/CDGPT2/ckpt-50


  0%|                                                                                          | 0/500 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
  8%|██████▋                                                                        | 42/500 [25:35<3:53:56, 30.65s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


 15%|███████████▌                                                                   | 73/500 [42:45<4:10:05, 35.14s/it]


KeyboardInterrupt: 